In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import string
import nltk
nltk.download('averaged_perceptron_tagger');

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option('display.max_columns', None)

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     EOF occurred in violation of protocol (_ssl.c:1129)>


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

In [3]:
# test.isna().sum()

In [4]:
# train[train.stage_status.isna() == False].head(20)

In [5]:
feat_1 = ['stage_status', 'agent_id']
feat_2 = ['subscriber_age', 'gender', 'city']
for data in (train, test):
    for feat in feat_1:
        data[feat] = data[feat].bfill().ffill()
    for feat_ in feat_2:
        data[feat_] = data[feat_].fillna(data[feat_].mode()[0])

In [6]:
# sns.set_style("white") 
# sns.countplot( y='label', data=train, palette="Set1")
# train[train.label == 'Backlkog']

In [7]:
# num_feats = list(train.select_dtypes(include=['int64', 'float64', 'int32']).columns)
# train[num_feats].hist(figsize=(20,15));

In [8]:
# test.subscriber_age > 90
# train = train.drop(train[train.subscriber_age > 90].index)

In [9]:
# sns.countplot(y = 'label', data = train, hue = "city", palette="Set1");

In [10]:
# train.info()

In [11]:
# train[train.city == 'ibadan']
train.city = train.city.str.replace('ibadan', 'Ibadan')

In [12]:
train = train.sort_values('date_time').reset_index(drop = True)
test = test.sort_values('date_time').reset_index(drop=True)

In [13]:
for data in (train, test):
    data['date_time'] = pd.to_datetime(data['date_time'])
    data['year'] = data.date_time.dt.year
    data['month'] = data.date_time.dt.month
    data['day'] = data.date_time.dt.day
    data['hour'] = data.date_time.dt.hour
    data['minute'] = data.date_time.dt.minute
    data['second'] = data.date_time.dt.second
    data['year_month_day'] = data.year.astype(str) + '_' + data.month.astype(str)  + '_' + data.day.astype(str)
    data['month_day'] = data.month.astype(str)  + '_' + data.day.astype(str)
    data.drop('date_time', axis = 1, inplace=True)

In [14]:
ID = test['ID']
test.drop('ID',inplace=True,axis=1)
train.drop('ID',inplace=True,axis=1)

In [15]:
[''.join(test[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(test))]

KeyboardInterrupt: 

In [184]:
# df_mashup = [''.join(df[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(df))]
# train_mashup = [''.join(train[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(train))]
test_mashup = [''.join(test[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(test))]

KeyboardInterrupt: 

In [150]:
df = pd.concat([train, test])
mashup = [''.join(df[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']].astype(str).iloc[i].values) for i in range(len(df))]
mashup_cleaned = [mashup[i].strip().replace(',','') for i in range(len(mashup))]
CouVec = CountVectorizer()
CouVec.fit(df['agent_id'])

train_words = pd.DataFrame(CouVec.transform(train['agent_id']).toarray())
test_words = pd.DataFrame(CouVec.transform(test['agent_id']).toarray())

In [152]:
# train_words

In [154]:
df[['subscriber_id', 'agent_id', 'entry_channel_id', 'stage_status', 'city']]

,subscriber_id,agent_id,entry_channel_id,stage_status,city
0,70264236-ad1e-4b8d-8402-d6ff24d863d8,87ca3dc9-ce2d-4b03-a72e-73d4b35b0739,4,schedule,Ibadan
1,70264236-ad1e-4b8d-8402-d6ff24d863d8,87ca3dc9-ce2d-4b03-a72e-73d4b35b0739,4,fail,Ibadan
2,70264236-ad1e-4b8d-8402-d6ff24d863d8,87ca3dc9-ce2d-4b03-a72e-73d4b35b0739,4,fail,Ibadan
3,baa48562-922a-4383-8564-77d03aa23339,87ca3dc9-ce2d-4b03-a72e-73d4b35b0739,4,fail,Ibadan
4,baa48562-922a-4383-8564-77d03aa23339,87ca3dc9-ce2d-4b03-a72e-73d4b35b0739,4,fail,Ibadan
...,...,...,...,...,...
92073,122a89e5-5947-4168-939a-90ce1aa17063,c36a4418-00ce-4560-9980-9aa9d56d9e64,4,pass,Oshogbo
92074,25898f3a-5110-48ce-b4ec-13be804a0f85,ee9027a1-2d57-4cf4-86ca-7b0be53f3bae,7,schedule,Ibadan
92075,6bb8fa1c-547a-4843-bdf9-9dc5e9567bde,11344293-5168-4f92-a7ce-0b94bbf17a94,4,schedule,Abeokuta
92076,2a35e7a1-7c6e-4dfc-b09b-ac2d75db623c,4f26dcf0-02e8-4651-87bc-3ca7ab4f4127,4,schedule,Abeokuta


In [128]:
LDA = LatentDirichletAllocation(n_components=15, max_iter=100, random_state=42)
LDA.fit(pd.concat([train_words, test_words]))
Topics = [f'Topic_{x}' for x in range(0,15)]
train[Topics] = LDA.transform(train_words)
test[Topics] = LDA.transform(test_words)

ValueError: Length of values (5) does not match length of index (214847)